# Transformers

this note book is here to help me refresh some of my understanding of the basic transformers architecture

we want to implement the encoder part of the architecture in [attention is all you need paper](https://arxiv.org/pdf/1706.03762):




architecture screentshot:

![](20251120024008.png)

My goal with be to go through one pass of transformer layer for a data, and try to explain each layer, finally I will convert this jupyter notebook to a python code and train it on a simple dataset

In [1]:
# I want this note book to be very simple so I will make the data very simple, i.e use whatever I have written till now as training data

training_data = list("""
# Transformers

this note book is here to help me refresh some of my understanding of the basic transformers architecture

we want to implement the encoder part of the architecture in [attention is all you need paper](https://arxiv.org/pdf/1706.03762):

My goal with be to go through one pass of transformer layer for a data, and try to explain each layer, finally I will convert this jupyter notebook to a python code and train it on a simple dataset

# I want this note book to be very simple so I will make the data very simple, i.e use whatever I have written till now as training data

""")

In [2]:
# I don't want to get too deep into tokenization for this notebook so I am just going to instead use all the unique characters
# present in the training data as distinct tokens
vocabulary_list = list(set(training_data))

In [3]:
print(vocabulary_list[:5])
print(len(vocabulary_list))

['T', 'n', 'm', 'v', 'k']
44


In [4]:
# let's create training and testing data
# training and testing data for next token prediction would look something like

# the way the transformer works is that for a single example sentence it trains the model for multiple token prediction
print(training_data[:9])

['\n', '#', ' ', 'T', 'r', 'a', 'n', 's', 'f']


In [5]:
# here if x is
training_data[:8]

['\n', '#', ' ', 'T', 'r', 'a', 'n', 's']

In [6]:
# then y would be
training_data[1:9]

['#', ' ', 'T', 'r', 'a', 'n', 's', 'f']

In [7]:
# ok before we make create training data we need to convert our tokens to a unique index to do that I will do
token_to_index = {c:i for i,c in enumerate(vocabulary_list)}
index_to_token = {i:c for i,c in enumerate(vocabulary_list)}

In [8]:
# now we let's convert our training data to a torch tensor
import torch

training_data_tensor = torch.tensor([token_to_index[c] for c in training_data], dtype=torch.long)

In [9]:
print(training_data_tensor[:10])
print([index_to_token[ix.item()] for ix in training_data_tensor[:10]])

tensor([13, 23, 36,  0, 39, 17,  1,  7, 34, 33])
['\n', '#', ' ', 'T', 'r', 'a', 'n', 's', 'f', 'o']


In [10]:
# now let's create training and testing set
block_size = 8
x = torch.stack([training_data_tensor[ix:ix+block_size] for ix in range(len(training_data_tensor)-block_size)] )
# max ix len(training_data_tensor)-block_size - 1
# so ix + block_size = len(training_data_tensor) - 1
# so final example won't include last character
y = torch.stack([training_data_tensor[ix:ix+block_size]for ix in range(1,len(training_data_tensor)-block_size+1)]) 



In [11]:
print("x training data")
print(x[:5])
print("y training data")
print(y[:5])

x training data
tensor([[13, 23, 36,  0, 39, 17,  1,  7],
        [23, 36,  0, 39, 17,  1,  7, 34],
        [36,  0, 39, 17,  1,  7, 34, 33],
        [ 0, 39, 17,  1,  7, 34, 33, 39],
        [39, 17,  1,  7, 34, 33, 39,  2]])
y training data
tensor([[23, 36,  0, 39, 17,  1,  7, 34],
        [36,  0, 39, 17,  1,  7, 34, 33],
        [ 0, 39, 17,  1,  7, 34, 33, 39],
        [39, 17,  1,  7, 34, 33, 39,  2],
        [17,  1,  7, 34, 33, 39,  2, 37]])


# Embedding Table

![](20251121001141.png)


This is a look up table between the vocabulary index and n dimensional vector,
during the training of transformer model this vectors also gets trained, i.e where these vectors point to gets updated,
based on the similarity between these vectors, if let's say I have 2 tokens "dog" and "pooch", during the start of training process
they might point in very different directions, but after the training both would point to pretty much same place

### Question?:

1. What is so special about the training process that transforms these vectors from pointing in random ass direction, to actually have some meaning
    * for now I am gonna assume that the answer is that the transformer architecture expects and assumes these vectors to be what I have described
    * and based on this assumption, the subsequent layers performs its operation, so optimizing the loss leads to these embedding vector looking more like actual high dimensional representation of the words 

In [12]:
from torch import nn

EMBEDDING_DIMENSION = 8
VOCAB_SIZE = len(vocabulary_list)

embeddings_table = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIMENSION)

In [13]:
# some experimentation on how embeddings table work,
print(embeddings_table(torch.tensor([[0,1,2,3]], dtype=torch.long)))
# it goes to each item in tensor and assumes each item is a index converts it to its corresponding embedding vector

tensor([[[ 0.6540, -1.0201,  0.3798,  0.4197, -0.9887,  1.0880, -0.6461,
          -0.3841],
         [-0.3048,  0.5368,  0.8345,  0.3198,  0.5887, -0.7464,  0.7392,
          -0.9602],
         [ 1.5515,  1.0876, -1.1028,  1.7605,  0.4289,  0.3081,  1.2073,
           0.4674],
         [-1.3129,  1.0818,  1.9786,  1.8031,  1.7061,  0.3980, -1.7115,
           0.4997]]], grad_fn=<EmbeddingBackward0>)


I want to do a very simple forward pass so I am gonna create my forward pass batch now

In [14]:
x_batch = x[:5]
y_batch = y[:5]

A question lingers, what does this (shifted right) mean:

![](20251121002201.png)

this just means that our input is shifted from the target output

In [15]:
x_embeddings = embeddings_table(x_batch)

In [16]:
# just one example
x_embeddings[:1]

tensor([[[-0.9678, -0.6803, -0.1128,  0.9726,  1.3120,  0.6363,  1.0772,
          -0.1022],
         [ 0.7778, -0.6351,  0.5538, -1.4203, -0.4150, -0.3149,  0.4821,
           0.0528],
         [-0.6133, -0.8224,  0.2207, -0.8314,  1.1423, -0.7206,  1.6457,
          -0.5046],
         [ 0.6540, -1.0201,  0.3798,  0.4197, -0.9887,  1.0880, -0.6461,
          -0.3841],
         [ 0.2969, -0.0529,  0.1083,  0.0809, -0.2940, -2.0801,  0.6963,
           2.3153],
         [-0.2635, -0.7460,  0.0114, -0.4884, -1.3530,  0.4391, -1.3136,
          -0.9813],
         [-0.3048,  0.5368,  0.8345,  0.3198,  0.5887, -0.7464,  0.7392,
          -0.9602],
         [ 1.1118,  2.3784,  1.9857, -0.7493, -0.2177,  1.6323, -0.5704,
          -1.0169]]], grad_fn=<SliceBackward0>)

# Positional Encoding

![](20251121135418.png)


From my past understanding this is sort of values with varies with the position of the token in the sequence to encode the information about the position of the token in the sequence

so for each position there will be a vector associated to it, which will get added to the original embedding vector at that position

### Questions?:
1. Why Add these vectors to the original embedding vector? Can it not be appended or create some other type of encoding create a new channel perhaps like we do for CNNs
    - Ans: The Idea behind adding these is how we treat embedding vectors, you can think of embedding vector as the original absolute meaning of a token, now depending on whether it appears at the beggining of a sentence or end of a sentence it's meaning might differ, i.e its embedding vector might change its position, that change is capture by the addition of this positional embeddding vector
2. Why do these needs to be a vector all together can these not be like a single number which gets added?
    - Ans: well a vector is a more generalized version of a single number, if single number is the right approach then expectation is that the network would train the embedings to become a single number

## Sinusoidal Encoding

![](20251121140214.png)

here d_model is the dimension of the embedding

In the original Paper they used a fix positional sinusoidal encoding, they mentioned the performance for both learned and not learned were identical, they wanted to experiment with sinusoidal encoding, because they wanted to test the model beyond the trained context length

# Question?:
1. But why sinusoidal encoding

In [17]:
positional_embedding_table = nn.Embedding(block_size, EMBEDDING_DIMENSION)

In [18]:
x_pos_embeddings = positional_embedding_table(torch.arange(x_embeddings.shape[1])) # C, E

x_embeddings # B, C, E

x_embeddings_total = x_embeddings + x_pos_embeddings # B, C, E + C, E -> pytorch checks the shape starting from right and if there is an extra dimension it creates a new dimention and copuies the same thing over, like C, E -> (1, C, E) -> (B, C, E)

# Self Attention Layer

![](20251123232032.png)

I will start of by explaining what this layer does in a high level, then I will dig deep into how it does this, initially I will go over a single head self attention, 
then understand myself and explain why multi head self attention

this is the 3b1b interpretation of this layer on a high level, which I found to be the most elegant

## The Explanation
This layer as a whole tells us how should the original embedding vector be modified, so that it's meaning is enriched with the context of the surrounding tokens, for example take the sentence:

" That blue aeroplane is very dangerous "

in this example initially "aeroplane"'s embedding vector would straight up point to the absolute aeroplane,
then attention layer outputs a result, that result when added to the original embedding vector, nudges the aeroplane's vector in a direction closer to blue and dangerous

that is on high level what this layer does, now going into the detail let's start by the equation

![](20251123233808.png)

the above represent the equation describing the self attention mechanism, for the purpose of this excercise we will focus on masked self attention

here Q, K, V are all matrices

Q being the query matrix, K Key matrix and V value matrix

let me do one thing and form this forumla in our on going example and then explain

In [19]:
d_k = d_q = 4

W_q = nn.Linear(EMBEDDING_DIMENSION, d_q, bias = False)
W_k = nn.Linear(EMBEDDING_DIMENSION, d_k, bias = False)
W_v = nn.Linear(EMBEDDING_DIMENSION, EMBEDDING_DIMENSION, bias = False) # for single head attention this needs to be same as embedding dimension cause the resulting vector of attention layer get added to input embedding vector so both need to have same dimension


In [20]:
W_q.weight.shape

torch.Size([4, 8])

In [21]:
x_embeddings_total.shape

torch.Size([5, 8, 8])

In [22]:
W_q.weight@x_embeddings_total[0][0]

tensor([-1.3344,  0.4420,  0.7100,  0.0166], grad_fn=<MvBackward0>)

In [23]:
W_q(x_embeddings_total)

tensor([[[-1.3344,  0.4420,  0.7100,  0.0166],
         [-0.2721,  0.0306,  0.4835, -0.7585],
         [-0.6923, -0.2566,  0.8054, -1.5229],
         [ 0.3071,  0.6705, -0.8253,  0.0543],
         [-0.0796,  0.4107, -0.4603,  0.1473],
         [ 0.5555, -0.0381, -0.8428,  0.9641],
         [ 0.1081, -0.3633,  0.3026, -0.3332],
         [ 1.9611, -0.9610, -1.2928, -0.7901]],

        [[-0.5917,  0.7240,  0.6131,  0.0352],
         [-1.3023,  0.0883,  1.0764, -0.7143],
         [ 0.8351, -0.6233, -0.5685, -1.5361],
         [-0.0424,  0.9604, -0.2519,  0.2954],
         [ 0.0658,  0.1161, -0.5336,  0.3131],
         [-0.0375,  0.1796, -0.7163,  0.9803],
         [ 1.3795, -0.8129, -0.2474, -0.5987],
         [ 2.1010, -1.1908, -1.9434, -1.1397]],

        [[-1.6220,  0.7817,  1.2060,  0.0793],
         [ 0.2251, -0.2784, -0.2974, -0.7274],
         [ 0.4857, -0.3334,  0.0049, -1.2950],
         [ 0.1030,  0.6658, -0.3252,  0.4613],
         [-0.5272,  0.3337, -0.4071,  0.3293],
         

In [24]:
# pass the existing vector through a trainable linear transformation

Q, K, V = W_q(x_embeddings_total), W_k(x_embeddings_total), W_v(x_embeddings_total)

In [25]:
print(Q.shape, K.shape, V.shape)

torch.Size([5, 8, 4]) torch.Size([5, 8, 4]) torch.Size([5, 8, 8])


In [26]:
attention_matrix = Q@K.transpose(1, 2) # transpose the 1st and the 2nd dimension not the  this is equivalent to Q[i]@K[i].transpose where i is each element of the batch

Let me try and explain what just happened above, let's take example of a single batch

In [27]:
print(Q[0][:3])
print(K[0][:3])

tensor([[-1.3344,  0.4420,  0.7100,  0.0166],
        [-0.2721,  0.0306,  0.4835, -0.7585],
        [-0.6923, -0.2566,  0.8054, -1.5229]], grad_fn=<SliceBackward0>)
tensor([[ 0.8543, -0.3525, -0.6205,  1.1208],
        [-1.2757,  0.9904, -0.7430, -0.1793],
        [-0.4841,  0.7368,  0.0912,  1.1490]], grad_fn=<SliceBackward0>)



well the traditional explanation is:

when the static embedding for a word is passed through these layers it extract specific feature pertaining to corresponding transformation

let's take an example: "The bank of the River"

Query -> transform the static embedding of bank to something like "I am a Noun needing a definition, need to know what am I a river bank, a financial bank, etc"

Key -> transforms the static embedding of River to something like "I am a nature related word, related to river"

Value -> transforms the embedding to actual meaning info, as their it might not be the first attention layer, if it is a second layer, it won't be the absolute meaning

but this never really sat with me completely, I was not able to understand this fully and it seemed pretty handwavy as to explaining what these layers really do, maybe I understand for Value vector but not for Query and Key vector

need to think and understand this properly and clearly once and for all, 
think, what would happen if no linear layer was present


ok let's say no linear layer was present, than the Q*K.T would just give me the cosine similarity between static embeddings of tokens

let's say the attention matrix is

`A`

here `A[i][j]` would be the cosine similarity between the ith token and jth token  (here when I say token I mean the embedding of token)

now let's see what would `A@V` do, here I am assuming `V` to is just the static emebdding


`A_softmax` -> is softmax function applied on `A` across each column (why column? cause weights are distributed across column see below)

```

V`[i] = V[0]*A_softmax[i][0] + V[1]*A_softmax[i][1] + V[2]*A_softmax[i][2] + ...

```

here `V[i]` represents the static embedding for the ith word, so `V[i]` is a vector, and ``V`[i]`` is the final vector after attention matrix multiplication

``V`[i]`` we can think of it as change or how the original word gets modified

so when taking the weighted sum it tells us, in direction of which static embedding should the input embedding move to the most and least (depending on the weight), so that it represents that modified vector would represent the meaning of the entire sentence, **now I see the poblem with just using static embedding, the problem is if we just use static embedding to compute cosine similarity it would nudge the input emebedding to this layer in the direction which is most similar to the word which we see in the sentence, which we don't always want**

side note: one more thing I learned is that these query and key vector live in a space which is smaller than the static embedding dimension, so it would help computationaly too

In [28]:
# now each element of this of shape context length x context length
attention_matrix.shape

torch.Size([5, 8, 8])

In [29]:
attention_matrix[0]

tensor([[-1.7177,  1.6095,  1.0554,  0.2348,  0.5997, -0.4474,  0.5952,  2.4707],
        [-1.3933,  0.1542, -0.6732,  0.0851,  0.7560,  0.9053, -0.0617,  0.9885],
        [-2.7076,  0.3037, -1.5305,  0.5694,  1.9139,  1.8045, -0.3095,  1.5724],
        [ 0.5990,  0.8758,  0.3325,  0.2339, -0.6572, -0.5182,  0.0061, -1.6865],
        [ 0.2379,  0.8239,  0.4684,  0.2683, -0.2902, -0.5360,  0.0890, -0.7527],
        [ 2.0915, -0.2931,  0.7340, -0.0650, -1.0895, -1.1573, -0.0226, -1.8711],
        [-0.3408, -0.6628, -0.6753, -0.0821,  0.4195,  0.6690, -0.1916,  0.3538],
        [ 1.9307, -2.3512, -2.6830,  0.3430,  0.0616,  1.5071, -1.3763, -4.5390]],
       grad_fn=<SelectBackward0>)

```

V`[i] = V[0]*A_softmax[i][0] + V[1]*A_softmax[i][1] + V[2]*A_softmax[i][2] + ...

```

now in self attention when we compute the change required for ith vector we don't want any token above ith token to be deciding how it changes, cause during inference we don't have that information at all so we would need to find a way so that above becomes

```

V`[i] = V[0]*A_softmax[i][0] + V[1]*A_softmax[i][1] + V[2]*A_softmax[i][2] + ... + V[i]*A_softmax[i][i]

```

so to do this we convert the lower triangular part equal infinity so that when we do softmax we get zero

In [30]:
print(x_batch[0])
print(y_batch[0])

tensor([13, 23, 36,  0, 39, 17,  1,  7])
tensor([23, 36,  0, 39, 17,  1,  7, 34])


In [31]:
torch.ones_like(attention_matrix[0])

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [32]:
torch.tril(torch.ones_like(attention_matrix[0]), diagonal=-1).bool() # should the diagonal also have no contribution?

tensor([[False, False, False, False, False, False, False, False],
        [ True, False, False, False, False, False, False, False],
        [ True,  True, False, False, False, False, False, False],
        [ True,  True,  True, False, False, False, False, False],
        [ True,  True,  True,  True, False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False],
        [ True,  True,  True,  True,  True,  True, False, False],
        [ True,  True,  True,  True,  True,  True,  True, False]])

In [33]:
attention_matrix[0].masked_fill(torch.tril(torch.ones_like(attention_matrix[0]), diagonal=-1).bool(), float('-inf'))

tensor([[-1.7177,  1.6095,  1.0554,  0.2348,  0.5997, -0.4474,  0.5952,  2.4707],
        [   -inf,  0.1542, -0.6732,  0.0851,  0.7560,  0.9053, -0.0617,  0.9885],
        [   -inf,    -inf, -1.5305,  0.5694,  1.9139,  1.8045, -0.3095,  1.5724],
        [   -inf,    -inf,    -inf,  0.2339, -0.6572, -0.5182,  0.0061, -1.6865],
        [   -inf,    -inf,    -inf,    -inf, -0.2902, -0.5360,  0.0890, -0.7527],
        [   -inf,    -inf,    -inf,    -inf,    -inf, -1.1573, -0.0226, -1.8711],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -0.1916,  0.3538],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -4.5390]],
       grad_fn=<MaskedFillBackward0>)

In [34]:
attention_matrix.masked_fill(torch.tril(torch.ones_like(attention_matrix), diagonal=-1).bool(), float('-inf'))[0]

tensor([[-1.7177,  1.6095,  1.0554,  0.2348,  0.5997, -0.4474,  0.5952,  2.4707],
        [   -inf,  0.1542, -0.6732,  0.0851,  0.7560,  0.9053, -0.0617,  0.9885],
        [   -inf,    -inf, -1.5305,  0.5694,  1.9139,  1.8045, -0.3095,  1.5724],
        [   -inf,    -inf,    -inf,  0.2339, -0.6572, -0.5182,  0.0061, -1.6865],
        [   -inf,    -inf,    -inf,    -inf, -0.2902, -0.5360,  0.0890, -0.7527],
        [   -inf,    -inf,    -inf,    -inf,    -inf, -1.1573, -0.0226, -1.8711],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -0.1916,  0.3538],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -4.5390]],
       grad_fn=<SelectBackward0>)

In [35]:
masked_attention_matrix = attention_matrix.masked_fill(torch.tril(torch.ones_like(attention_matrix), diagonal=-1).bool(), float('-inf'))

In [36]:
masked_attention_matrix[0]

tensor([[-1.7177,  1.6095,  1.0554,  0.2348,  0.5997, -0.4474,  0.5952,  2.4707],
        [   -inf,  0.1542, -0.6732,  0.0851,  0.7560,  0.9053, -0.0617,  0.9885],
        [   -inf,    -inf, -1.5305,  0.5694,  1.9139,  1.8045, -0.3095,  1.5724],
        [   -inf,    -inf,    -inf,  0.2339, -0.6572, -0.5182,  0.0061, -1.6865],
        [   -inf,    -inf,    -inf,    -inf, -0.2902, -0.5360,  0.0890, -0.7527],
        [   -inf,    -inf,    -inf,    -inf,    -inf, -1.1573, -0.0226, -1.8711],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -0.1916,  0.3538],
        [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf, -4.5390]],
       grad_fn=<SelectBackward0>)

In [38]:
import torch.nn.functional as F


F.softmax(masked_attention_matrix[0], dim=-2)

tensor([[1.0000e+00, 8.1082e-01, 7.9817e-01, 2.3487e-01, 1.5183e-01, 5.9837e-02,
         2.4539e-01, 5.4818e-01],
        [0.0000e+00, 1.8918e-01, 1.4170e-01, 2.0223e-01, 1.7753e-01, 2.3143e-01,
         1.2722e-01, 1.2452e-01],
        [0.0000e+00, 0.0000e+00, 6.0127e-02, 3.2823e-01, 5.6508e-01, 5.6880e-01,
         9.9297e-02, 2.2327e-01],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 2.3467e-01, 4.3200e-02, 5.5749e-02,
         1.3615e-01, 8.5798e-03],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 6.2358e-02, 5.4762e-02,
         1.4791e-01, 2.1828e-02],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.9421e-02,
         1.3230e-01, 7.1340e-03],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.1173e-01, 6.6003e-02],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 4.9507e-04]], grad_fn=<SoftmaxBackward0>)

### should the diagonal (i.e the current token) have any contribution towards update of input embedding vector?

if it does then ``V`[0] = V[0]`` and that would mean the updated embedding would become ``E_updated[0] = V[0] + E[0]`` need to think more about it

but according to my understanding `V[0]` is same as the absolute embedding of the token so `V[0] + E[0]` would be pracitically `2*absolute_mebdding[0]` since even `E[0]` was created any info about token after that so just itself, but since we don layer norm `norm(2*absolute_embedding[0]) = absolute_embedding[0]` so even if it scales a lot layer normalization brings down the scale of embedding vector so it fits with rest of embedding vector's scale

In [39]:
batch_attention_matrix = F.softmax(masked_attention_matrix, dim=-2)

In [ ]:
batch_attention_matrix[0]

tensor([[1.0000e+00, 8.1082e-01, 7.9817e-01, 2.3487e-01, 1.5183e-01, 5.9837e-02,
         2.4539e-01, 5.4818e-01],
        [0.0000e+00, 1.8918e-01, 1.4170e-01, 2.0223e-01, 1.7753e-01, 2.3143e-01,
         1.2722e-01, 1.2452e-01],
        [0.0000e+00, 0.0000e+00, 6.0127e-02, 3.2823e-01, 5.6508e-01, 5.6880e-01,
         9.9297e-02, 2.2327e-01],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 2.3467e-01, 4.3200e-02, 5.5749e-02,
         1.3615e-01, 8.5798e-03],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 6.2358e-02, 5.4762e-02,
         1.4791e-01, 2.1828e-02],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.9421e-02,
         1.3230e-01, 7.1340e-03],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.1173e-01, 6.6003e-02],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 4.9507e-04]], grad_fn=<SelectBackward0>)

In [41]:
V[0]

tensor([[-1.1122,  0.2670,  1.0829,  0.1845,  1.0239, -0.7114,  0.0507,  1.2775],
        [-1.2197,  1.8542,  0.9127,  1.1249,  0.7362,  0.9720,  1.1122, -0.4257],
        [-1.2100, -0.2942, -0.4787, -1.1567,  0.7780,  0.1016,  0.1374, -0.0164],
        [ 0.4472, -0.8412, -1.3301,  0.4146,  0.2441,  0.0365,  1.0123,  0.9113],
        [-0.0561,  0.5671, -0.0635,  0.2287,  1.2057, -0.8641, -1.1553,  0.1430],
        [ 0.5620, -0.1501, -0.0190,  1.1640, -0.6168,  0.9801,  0.3207,  0.2334],
        [-0.0490,  1.1685,  0.7689,  0.0787, -0.1436,  0.1231, -0.2789, -0.8553],
        [ 2.0064,  0.8065, -1.1616, -0.1256, -1.8439,  1.2658, -0.3119, -1.7680]],
       grad_fn=<SelectBackward0>)

In [42]:
batch_attention_matrix[0]@V[0]

tensor([[-1.8489e+00,  2.1440e+00,  6.6959e-01,  3.2553e-01,  1.3992e+00,
          8.1799e-01,  9.0427e-01, -1.0161e-02],
        [ 5.1969e-02,  4.5400e-01, -2.2661e-01,  4.3711e-01,  1.2232e-01,
          4.5238e-01,  2.2939e-01, -1.4813e-01],
        [ 8.0513e-01,  2.3737e-01, -6.9501e-01,  8.3765e-01,  3.1477e-02,
          3.8211e-01, -2.2728e-01,  3.2017e-02],
        [ 1.4442e-01, -1.5270e-02, -2.2121e-01,  1.8171e-01,  3.9625e-02,
          5.3503e-02,  1.6487e-01,  1.0143e-01],
        [ 6.3835e-02,  2.1758e-01,  8.3382e-02,  8.6903e-02, -2.0076e-02,
          4.5623e-02, -1.0255e-01, -1.4340e-01],
        [ 2.4373e-02,  1.5592e-01,  9.2883e-02,  4.3759e-02, -5.0295e-02,
          5.4150e-02, -2.9692e-02, -1.1890e-01],
        [ 1.2696e-01,  1.8379e-01,  9.2405e-03,  5.0095e-04, -1.3774e-01,
          9.7299e-02, -5.1750e-02, -2.1225e-01],
        [ 9.9333e-04,  3.9930e-04, -5.7509e-04, -6.2175e-05, -9.1286e-04,
          6.2667e-04, -1.5441e-04, -8.7530e-04]], grad_fn=<MmBack

In [44]:
V_prime = batch_attention_matrix@V

In [46]:
embedding_unnormalized = V_prime + x_embeddings_total

# Layer normalization

we just normalize across the embedding dimension to make sure this addition does not scale the embedding vector to much across each layer so that it has the scale of the original embedding vector

In [47]:
mean_emebdding = embedding_unnormalized.mean(dim=-1)
mean_emebdding.shape

torch.Size([5, 8])

In [51]:
std_embedding = embedding_unnormalized.std(dim=-1)
std_embedding.shape

torch.Size([5, 8])

In [52]:
embedding_unnormalized.shape

torch.Size([5, 8, 8])

In [53]:
print(mean_emebdding[0][0], std_embedding[0][0])

tensor(0.8209, grad_fn=<SelectBackward0>) tensor(2.0766, grad_fn=<SelectBackward0>)


In [59]:
(embedding_unnormalized[0][0] - mean_emebdding[0][0])/(std_embedding[0][0] + 0.0001)

tensor([-1.5510,  0.6891, -0.5757,  0.2310,  1.1508,  0.3524,  0.9104, -1.2070],
       grad_fn=<DivBackward0>)

In [65]:
embedding_normalized = (embedding_unnormalized - mean_emebdding.unsqueeze(-1))/(std_embedding.unsqueeze(-1) + 0.0001) # unsqueeze needed to broadcast
embedding_normalized[0][0]

tensor([-1.5510,  0.6891, -0.5757,  0.2310,  1.1508,  0.3524,  0.9104, -1.2070],
       grad_fn=<SelectBackward0>)